In [1]:
import numpy as np
import pandas as pd
from numpy.random import random_sample as rs
from sasmodels.data import load_data
from sasmodels.data import set_beam_stop
from sasmodels.data import empty_data1D
from sasmodels.core import load_model
from sasmodels.direct_model import DirectModel, call_kernel
from matplotlib import pyplot as plt
import pandas
from matplotlib import pyplot as plt
import string

# Simulation Functions
These are a series of helper functions that simulate a batch of data for a given morphology. To simulate data of a different morphology, it would be required to add a new function to this section calling the appropriate sas model, defined is sasmodels.
The parameter ranges for each model are defined in dictionaries of parameters.
|parameter|(min, max)|
|:-----------|----------:|
|polydispersity|0.1 -0.35|
|Scale|1|
|SLD (body)|1 - 10|
|SLD (shell)|4 &plusmn; (.4, 3.5)|

In [2]:
shared_params = { 'polydispersity_min': .1,   #These are the minimum and maximum parameters shared by all models
                  'polydispersity_max': .35,  #The are defined as a minimum and maximum value
                  'scale_min': 1.,
                  'scale_max': 1.,
                  'background_min': .001,
                  'background_max': .001,
                  'sld_body_min': 1,
                  'sld_body_max': 10,
                  'sld_body_factor_min':2,
                  'sld_body_factor_max':4,
                  'sld_body_difference_min':.5,
                  'sld_body_difference_max':3.5,
                  'sld_solvent_min': 4.,
                  'sld_solvent_max': 4.,
                  'polydispersity_count': 3,
                  'scale_count': 30,
                  'background_count': 3,
                  'sld_body_count': 3,
                  'sld_body_factor_count':3,
                  'sld_solvent_count': 3}  

def minmax(q):
   minval = 2*np.pi/np.max(q.astype(float))
   maxval = 2*np.pi/np.min(q.astype(float))
   return(minval, maxval)


## cylinder

|parameter|(min,max)|
|---:|:---|
|aspect_ratio|(2.0,16.0)|
|background|(1e-3,1e-3)|
|contrast|(0.5,3.5)|
|length|(101.2,1696.3)|
|polydispersity|(0.1,0.3)|
|radius|(12.1,298.7)|
|scale|(1.0,1.0)|
|sld_core|(0.5,7.5)|
|sld_solvent|(4.0,4.0)|
|shell_ratio|(0.2,0.9)|

In [3]:
cylp = {'radius_min': 20., #20
        'radius_max':500.,
        'radius_count': 15,
        'radius_log_min': 1.03,
        'radius_log_max': 2.70,
        'length_factor_min': 4., #8 + return to 3
        'length_factor_max': 16.,
        'aspect_ratio_min':8.,
        'aspect_ratio_max':32.,
        'length_min': 100.,
        'length_max': 1700.,
        'length_log_min': 2.,
        'length_log_max':3.23,
        'length_factor_count':15}


def make_cylinder_test(q, num_vals, ar_min = None, ar_max = None):
   minv, maxv = minmax(q)
   model = load_model('cylinder')
   kernel = model.make_kernel([q])
   length_log_vals = (cylp['length_log_max'] - cylp['length_log_min']) * rs(num_vals) + cylp['length_log_min']
   if ar_min is None or ar_max is None:
      aspect_ratio_vals = (cylp['aspect_ratio_max'] - cylp['aspect_ratio_min']) * rs(num_vals) + cylp['aspect_ratio_min']
   else:
      log_aspect_ratio_vals = (ar_max - ar_min) * rs(num_vals) + ar_min
      aspect_ratio_vals = 2**log_aspect_ratio_vals
   length_vals = 10**length_log_vals
   radius_vals = length_vals/(2*aspect_ratio_vals)
   morph_vals = np.array(['cylinder' for i in range(num_vals)])
   polydispersity_vals = (shared_params['polydispersity_max'] - shared_params['polydispersity_min']) * rs(num_vals) + shared_params['polydispersity_min']
   sld_body_difference_vals = (shared_params['sld_body_difference_max'] - shared_params['sld_body_difference_min']) * rs(num_vals) + shared_params['sld_body_difference_min']
   sld_solvent_range = (shared_params['sld_solvent_max'] - shared_params['sld_solvent_min']) * rs(num_vals) + shared_params['sld_solvent_min']
   scale_range = (shared_params['scale_max'] - shared_params['scale_min']) * rs(num_vals) + shared_params['scale_min']
   background_range = (shared_params['background_max'] - shared_params['background_min']) * rs(num_vals) + shared_params['background_min']
   sld_body_range = sld_solvent_range-(sld_body_difference_vals*np.random.choice([1,-1], size=num_vals))
   
   cylinder_dict = {'radius': radius_vals, 'length': length_vals, 'polydispersity': polydispersity_vals, 'sld_core': sld_body_range, 'sld_solvent': sld_solvent_range, 'scale': scale_range, 'background': background_range, 'contrast':sld_body_difference_vals, 'morphology': morph_vals, 'aspect_ratio': aspect_ratio_vals}
   index = 0
   valid = np.zeros(radius_vals.shape[0])
   for qi in q:
      cylinder_dict[qi] = np.zeros(num_vals)
   for index in range(num_vals):
      rad = radius_vals[index]
      length = length_vals[index]
      pd = polydispersity_vals[index]
      scale = scale_range[index]
      background = background_range[index]
      sld_body = sld_body_range[index]
      sld_solvent = sld_solvent_range[index]
      if 2*rad<maxv and 2*rad>minv and length<maxv and length>minv and length/rad>cylp['length_factor_min']:
         pars = {'length': length, 'length_pd' :pd, 'length_pd_type': 'schulz', 'length_pd_n': 40, 'length_pd_nsigma': 3,
            'radius': rad, 
            'background': background,
            'sld': sld_body,
            'sld_solvent': sld_solvent,
            'scale': scale}
         Iq = call_kernel(kernel, pars)
         for qi in range(q.shape[0]):
            cylinder_dict[q[qi]][index] = Iq[qi]
         valid[index]=1
   for key, val in cylinder_dict.items():
      cylinder_dict[key] = val[np.where(valid.astype(int))[0]]
   cylinder_df = pandas.DataFrame(cylinder_dict)
   return(cylinder_df)


## Disk

|parameter|(min,max)|
|---:|:---|
|aspect_ratio|(2.0,15.9)|
|background|(1e-3,1e-3)|
|contrast|(0.5,3.5)|
|length|(24.2,432.6)|
|polydispersity|(0.1,0.3)|
|radius|(79.6,500.7)|
|scale|(1.0,1.0)|
|sld_core|(0.5,7.5)|
|sld_solvent|(4.0,4.0)|
|shell_ratio|(0.2,0.9)|

In [4]:
diskp = {'radius_min': 20.,
        'radius_max':850.,
        'radius_count': 15,
        'length_min': 25.,
        'length_max': 425.,
        'radius_log_min': 1.9,
        'radius_log_max': 2.70,
        'length_factor_min': .02, #10
        'length_factor_max':.25, #Compare from 1/4 - return to .35
        'aspect_ratio_min':8.,
        'aspect_ratio_max':32.,
        'length_factor_count':15}

def make_disk_test(q, num_vals, ar_min = None, ar_max = None):
   minv, maxv = minmax(q)
   model = load_model('cylinder')
   kernel = model.make_kernel([q])
   radius_log_vals = (diskp['radius_log_max'] - diskp['radius_log_min']) * rs(num_vals) + diskp['radius_log_min']
   if ar_min is None or ar_max is None:
      aspect_ratio_vals = (diskp['aspect_ratio_max'] - diskp['aspect_ratio_min']) * rs(num_vals) + diskp['aspect_ratio_min']
   else:
      log_aspect_ratio_vals = (ar_max - ar_min) * rs(num_vals) + ar_min
      aspect_ratio_vals = 2**log_aspect_ratio_vals
   polydispersity_vals = (shared_params['polydispersity_max'] - shared_params['polydispersity_min']) * rs(num_vals) + shared_params['polydispersity_min']
   sld_body_difference_vals = (shared_params['sld_body_difference_max'] - shared_params['sld_body_difference_min']) * rs(num_vals) + shared_params['sld_body_difference_min']
   radius_vals = 10**radius_log_vals
   diameter_vals = 2*radius_vals
   length_vals = diameter_vals/aspect_ratio_vals
   scale_range = (shared_params['scale_max'] - shared_params['scale_min']) * rs(num_vals) + shared_params['scale_min']
   background_range = (shared_params['background_max'] - shared_params['background_min']) * rs(num_vals) + shared_params['background_min']
   sld_solvent_range = (shared_params['sld_solvent_max'] - shared_params['sld_solvent_min']) * rs(num_vals) + shared_params['sld_solvent_min']
   sld_body_range = sld_solvent_range - (sld_body_difference_vals*np.random.choice([1,-1], size=num_vals))
   disk_dict = {'radius': radius_vals, 'length': length_vals, 'polydispersity': polydispersity_vals, 'sld_core': sld_body_range, 'sld_solvent': sld_solvent_range, 'scale': scale_range, 'background': background_range, 'contrast':np.abs(sld_body_difference_vals), 'aspect_ratio': aspect_ratio_vals}
   index = 0
   valid = np.zeros(num_vals)
   for qi in q:
      disk_dict[qi] = np.zeros(num_vals)
   for index in range(num_vals):
      rad = radius_vals[index]
      length = length_vals[index]
      pd = polydispersity_vals[index]
      scale = scale_range[index]
      background = background_range[index]
      sld_body = sld_body_range[index]
      sld_solvent = sld_solvent_range[index]
      if (length < maxv and length > minv) and (2*rad < maxv and 2*rad > minv):
         pars = {'radius': rad, 'radius_pd' :pd, 'radius_pd_type': 'schulz', 'radius_pd_n': 40, 'radius_pd_nsigma': 3,
            'length': length,
            'background': background,
            'sld': sld_body,
            'sld_solvent': sld_solvent,
            'scale': scale}
         Iq = call_kernel(kernel, pars)
         for qi in range(q.shape[0]):
            disk_dict[q[qi]][index] = Iq[qi]
         valid[index]=1
   for key, val in disk_dict.items():
      disk_dict[key] = val[np.where(valid.astype(int))[0]]
   disk_df = pandas.DataFrame(disk_dict)
   return(disk_df)

## Sphere

|parameter|(min, max)|
|---:|:---|
|background|(1e-3,1e-3)|
|contrast|(0.5,3.5)|
|polydispersity|(0.1,0.3)|
|radius|(40.0,464.0)|
|scale|(1.0,1.0)|
|sld_core|(0.5,7.5)|
|sld_solvent|(4.0,4.0)|
|shell_ratio|(0.2,0.9)|
|aspect_ratio|(2.0,16.0)|

In [5]:
spherep = {'radius_min': 20., #20
           'radius_max':850,
           'radius_log_min': 1.03,
           'radius_log_max': 2.70,
           'radius_count':225}

def make_sphere_test(q, num_vals, rad_min = None, rad_max = None):
   minv, maxv = minmax(q)
   model = load_model('sphere')
   kernel = model.make_kernel([q])
   if rad_min is None:
      radius_vals = (spherep['radius_max']-spherep['radius_min'])*rs(num_vals)+spherep['radius_min']
   else:
      log_rad_vals = (rad_max-rad_min)*rs(num_vals)+rad_min
      radius_vals = 2**log_rad_vals
   polydispersity_vals = (shared_params['polydispersity_max'] - shared_params['polydispersity_min']) * rs(num_vals) + shared_params['polydispersity_min']
   sld_body_difference_vals = (shared_params['sld_body_difference_max'] - shared_params['sld_body_difference_min']) * rs(num_vals) + shared_params['sld_body_difference_min']
   scale_range = (shared_params['scale_max'] - shared_params['scale_min']) * rs(num_vals) + shared_params['scale_min']
   background_range = (shared_params['background_max'] - shared_params['background_min']) * rs(num_vals) + shared_params['background_min']
   sld_solvent_range = (shared_params['sld_solvent_max'] - shared_params['sld_solvent_min']) * rs(num_vals) + shared_params['sld_solvent_min']
   sld_body_range = sld_solvent_range-(sld_body_difference_vals*np.random.choice([-1,1],size=num_vals))
   sphere_dict = {'radius': radius_vals, 'polydispersity': polydispersity_vals, 'sld_core': sld_body_range, 'sld_solvent': sld_solvent_range, 'scale': scale_range, 'background': background_range, 'contrast':sld_body_difference_vals}
   index = 0
   valid = np.zeros(num_vals)
   for qi in q:
      sphere_dict[qi] = np.zeros(num_vals)
   for index in range(num_vals):
      rad = radius_vals[index]
      pd = polydispersity_vals[index]
      scale = scale_range[index]
      background = background_range[index]
      sld_body = float(sld_body_range[index])
      sld_solvent = sld_solvent_range[index]
      if 2*rad < maxv and 2*rad > minv:
         pars = {'radius': rad, 'radius_pd' :pd, 'radius_pd_type': 'schulz', 'radius_pd_n': 40, 'radius_pd_nsigma': 3,
            'background': background,
            'sld': sld_body,
            'sld_solvent': sld_solvent,
            'scale': scale}
         Iq = call_kernel(kernel, pars)
         for qi in range(q.shape[0]):
            sphere_dict[q[qi]][index] = Iq[qi]
         valid[index]=1
   for key, val in sphere_dict.items():
      sphere_dict[key] = val[np.where(valid.astype(int))[0]]
   sphere_df = pandas.DataFrame(sphere_dict)
   return(sphere_df)

## Core-shell cylinder
 This function is used to simulate core-shell cylinders and this table includes the cs_cylinder specific parameters and ranges.

 |parameter|(min, max)|
 |-----:|:-------|
 |index|(0.0,2463.0)|
|radius|(49.2,293.9)|
|shell|(24.2,258.2)|
|length|(355.6,1698.1)|
|shell_ratio|(0.2,0.9)|
|core|(24.2,194.7)|
|polydispersity|(0.1,0.3)|
|sld_core|(-2.7,10.6)|
|sld_solvent|(4.0,4.0)|
|sld_shell|(0.5,7.5)|
|scale|(1.0,1.0)|
|background|(0.0,0.0)|
|contrast|(0.5,3.5)|
|contrast_2|(0.5,3.5)|
|aspect_ratio|(2.0,15.9)|
 

In [6]:
cs_cylinderp = {'radius_min':25.,
               'radius_max': 100.,
              'radius_log_min': 1.03,
              'radius_log_max': 2.,
              'length_log_min': 2.55,
              'length_log_max':3.23,
              'shell_min': .1,
              'shell_max': .95,
              'core_min': 20.,
              'core_max': 850.,
              'length_factor_min': 4., #8 + return to 3
              'length_factor_max': 16.,
              'length_min': 50,
              'length_max': 1700,
              'aspect_ratio_min':8.,
              'aspect_ratio_max':32.,
              'sld_shell_difference_min': 0.5,
              'sld_shell_difference_max': 3.5,
              'sld_shell_factor_min': .1,
              'sld_shell_factor_max': 2}

def make_core_shell_cylinder_test(q, num_vals, ar_min = None, ar_max = None, sr_min=None, sr_max = None, batch_size=1000):
   minv, maxv = minmax(q)
   model = load_model('core_shell_cylinder')
   kernel = model.make_kernel([q])
   length_log_vals = (cs_cylinderp['length_log_max']-cs_cylinderp['length_log_min'])*rs(num_vals)+cs_cylinderp['length_log_min']
   if sr_min is None:
      sr_min = cs_cylinderp['shell_min']
   if sr_max is None:
      sr_max = cs_cylinderp['shell_max']
   shell_ratio_vals = (sr_max-sr_min)*rs(num_vals)+sr_min
   if ar_min is None or ar_max is None:
      aspect_ratio_vals = (cs_cylinderp['aspect_ratio_max']-cs_cylinderp['aspect_ratio_min'])*rs(num_vals)+cs_cylinderp['aspect_ratio_min']
   else:
      log_aspect_ratio_vals = (ar_max-ar_min)*rs(num_vals)+ar_min
      aspect_ratio_vals = 2**log_aspect_ratio_vals
   length_vals = 10**length_log_vals
   radius_vals = length_vals/(2*aspect_ratio_vals)
   shell_vals = shell_ratio_vals*radius_vals
   core_vals = (1-shell_ratio_vals)*radius_vals
   polydispersity_vals = (shared_params['polydispersity_max'] - shared_params['polydispersity_min']) * rs(num_vals) + shared_params['polydispersity_min']
   scale_range = (shared_params['scale_max'] - shared_params['scale_min']) * rs(num_vals) + shared_params['scale_min']
   background_range = (shared_params['background_max'] - shared_params['background_min']) * rs(num_vals) + shared_params['background_min']
   sld_solvent_range = (shared_params['sld_solvent_max'] - shared_params['sld_solvent_min']) * rs(num_vals) + shared_params['sld_solvent_min']
   sld_shell_difference_vals = (cs_cylinderp['sld_shell_difference_max'] - cs_cylinderp['sld_shell_difference_min']) * rs(num_vals) + cs_cylinderp['sld_shell_difference_min']
   sld_body_difference_vals = (shared_params['sld_body_difference_max'] - shared_params['sld_body_difference_min']) * rs(num_vals) + shared_params['sld_body_difference_min']
   sld_shell_range = sld_solvent_range - (sld_shell_difference_vals*np.random.choice([1,-1],size=num_vals))
   sld_body_range = sld_shell_range - (sld_body_difference_vals*np.random.choice([1,-1],size=num_vals))
   morph_vals = np.array(['cs_cylinder' for i in range(num_vals)])
   batch_num = 0
   sphere_dict = {'radius': radius_vals, 'shell': shell_vals, 'length': length_vals, 'shell ratio': shell_ratio_vals, 'core': core_vals, 'polydispersity': polydispersity_vals, 'sld_core': sld_body_range, 'sld_solvent': sld_solvent_range, 'sld_shell': sld_shell_range, 'scale': scale_range, 'background': background_range, 'contrast': sld_shell_difference_vals, 'contrast_2': sld_body_difference_vals, 'morphology': morph_vals, 'aspect_ratio': aspect_ratio_vals}
   index = 0
   valid = np.zeros(num_vals)
   for qi in q:
      sphere_dict[qi] = np.zeros(num_vals)
   for index in range(num_vals):
      rad = core_vals[index]
      length = length_vals[index]
      shell = shell_vals[index]
      pd = polydispersity_vals[index]
      scale = scale_range[index]
      background = background_range[index]
      sld_body = sld_body_range[index]
      sld_solvent = sld_solvent_range[index]
      sld_shell = sld_shell_range[index]
      if length < maxv and length > minv and rad > minv and shell > minv and 2*(rad+shell)<maxv and length/rad > cs_cylinderp['length_factor_min']:
         pars = {'length': length, 'length_pd' :pd, 'length_pd_type': 'schulz', 'length_pd_n': 40, 'length_pd_nsigma': 3,
            'radius': rad,
            'thickness': shell,
            'sld_shell': sld_shell,
            'background': background,
            'sld_core': sld_body,
            'sld_solvent': sld_solvent,
            'scale': scale}
         Iq = call_kernel(kernel, pars)
         for qi in range(q.shape[0]):
            sphere_dict[q[qi]][index] = Iq[qi]
         valid[index]=1
   for key, val in sphere_dict.items():
      sphere_dict[key] = val[np.where(valid.astype(int))[0]]
   sphere_df = pandas.DataFrame(sphere_dict)
   return(sphere_df)


## Core-shell disk

|parameter|(min, max)|
|---:|:---|
|radius|(250.7,848.8)|
|shell|(24.2,233.8)|
|length|(73.4,765.6)|
|shell_ratio|(0.2,0.9)|
|core|(24.2,408.2)|
|polydispersity|(0.1,0.3)|
|sld_core|(-2.9,10.8)|
|sld_solvent|(4.0,4.0)|
|sld_shell|(0.5,7.5)|
|scale|(1.0,1.0)|
|background|(1e-3,1e-3)|
|contrast_2|(0.5,3.5)|
|contrast|(0.5,3.5)|
|aspect_ratio|(2.0,16.0)|

In [7]:
cs_diskp = {'radius_min': 250.,
        'radius_max':850.,
        'radius_count': 15,
        'radius_log_min': 1.03,
        'radius_log_max': 2.70,
        'aspect_ratio_min':8.,
        'aspect_ratio_max':32.,
        'length_min': 25,
        'length_max': 425,
        'shell_min': .1,
        'shell_max': .95,
        'sld_shell_difference_min': .5,
        'sld_shell_difference_max': 3.5,
        'sld_shell_factor_min': .1,
        'sld_shell_factor_max': 2.}

def make_core_shell_disk_test(q, num_vals, ar_min = None, ar_max = None, sr_min = None, sr_max=None):
   minv, maxv = minmax(q)
   model = load_model('core_shell_cylinder')
   kernel = model.make_kernel([q])
   radius_vals = (cs_diskp['radius_max']-cs_diskp['radius_min'])*rs(num_vals)+cs_diskp['radius_min']
   if ar_min is None or ar_max is None:
      aspect_ratio_vals = (cs_diskp['aspect_ratio_max']-cs_diskp['aspect_ratio_min'])*rs(num_vals)+cs_diskp['aspect_ratio_min']
   else:
      log_aspect_ratio_vals = (ar_max-ar_min)*rs(num_vals)+ar_min
      aspect_ratio_vals = 2**log_aspect_ratio_vals
   if sr_min is None:
      sr_min = cs_diskp['shell_min']
   if sr_max is None:
      sr_max = cs_diskp['shell_max']
   shell_ratio_vals = (sr_max-sr_min)*rs(num_vals)+sr_min
   length_vals = (2*radius_vals)/aspect_ratio_vals
   shell_vals = shell_ratio_vals*.5*length_vals
   core_vals = (1-shell_ratio_vals)*length_vals
   polydispersity_vals = (shared_params['polydispersity_max'] - shared_params['polydispersity_min']) * rs(num_vals) + shared_params['polydispersity_min']
   scale_range = (shared_params['scale_max'] - shared_params['scale_min']) * rs(num_vals) + shared_params['scale_min']
   background_range = (shared_params['background_max'] - shared_params['background_min']) * rs(num_vals) + shared_params['background_min']
   sld_body_difference_vals = (shared_params['sld_body_difference_max'] - shared_params['sld_body_difference_min']) * rs(num_vals) + shared_params['sld_body_difference_min']
   sld_solvent_range = (shared_params['sld_solvent_max'] - shared_params['sld_solvent_min']) * rs(num_vals) + shared_params['sld_solvent_min']
   sld_shell_difference_vals = (cs_diskp['sld_shell_difference_max']-cs_diskp['sld_shell_difference_min'])*rs(num_vals)+cs_diskp['sld_shell_difference_min']
   sld_shell_range = sld_solvent_range-(sld_shell_difference_vals*np.random.choice([-1,1],size=num_vals))
   sld_body_range = sld_shell_range-(sld_body_difference_vals*np.random.choice([-1,1],size=num_vals))
   sphere_dict = {'radius': radius_vals, 'shell': shell_vals, 'length': length_vals, 'shell ratio': shell_ratio_vals, 'core': core_vals, 'polydispersity': polydispersity_vals, 'sld_core': sld_body_range, 'sld_solvent': sld_solvent_range, 'sld_shell': sld_shell_range, 'scale': scale_range, 'background': background_range, 'contrast_2': sld_body_difference_vals, 'contrast': sld_shell_difference_vals, 'aspect_ratio':aspect_ratio_vals}
   index = 0
   valid = np.zeros(num_vals)
   for qi in q:
      sphere_dict[qi] = np.zeros(num_vals)
   for index in range(num_vals):
      core = core_vals[index]
      shell = shell_vals[index]
      rad = radius_vals[index]-shell
      pd = polydispersity_vals[index]
      scale = scale_range[index]
      background = background_range[index]
      sld_body = sld_body_range[index]
      sld_solvent = sld_solvent_range[index]
      sld_shell = sld_shell_range[index]
      if 2*(rad+shell) < maxv and rad > minv and shell>minv and np.abs(sld_shell-sld_solvent)>0.1 and np.abs(sld_shell-sld_body)>0.1 and core > minv:
         pars = {'radius': rad, 'radius_pd' :pd, 'radius_pd_type': 'schulz', 'radius_pd_n': 40, 'radius_pd_nsigma': 3,
            'length': core,
            'thickness': shell,
            'sld_shell': sld_shell,
            'background': background,
            'sld_core': sld_body,
            'sld_solvent': sld_solvent,
            'scale': scale}
         Iq = call_kernel(kernel, pars)
         for qi in range(q.shape[0]):
            sphere_dict[q[qi]][index] = Iq[qi]
         valid[index]=1
   for key, val in sphere_dict.items():
      sphere_dict[key] = val[np.where(valid.astype(int))[0]]
   sphere_df = pandas.DataFrame(sphere_dict)
   return(sphere_df)

## Core-shell sphere

|parameter|(min, max)|
|---:|:---|
|radius|(40.6,469.1)|
|shell|(24.2,378.5)|
|shell_ratio|(0.2,0.9)|
|core|(12.1,374.2)|
|polydispersity|(0.1,0.3)|
|sld_core|(-2.9,10.9)|
|sld_solvent|(4.0,4.0)|
|sld_shell|(0.5,7.5)|
|scale|(1.0,1.0)|
|background|(1e-3,1e-3)|
|contrast|(0.5,3.5)|
|contrast_2|(0.5,3.5)|
|aspect_ratio|(2.0,15.9)|

In [8]:
cs_spherep = {'radius_min':200., #Should be 20.
              'radius_max': 850.,
              'radius_log_min': 1.03,
              'radius_log_max': 2.70,
              'shell_min': .1,
              'shell_max': .95,
              'core_min': 20,
              'core_max': 850,
              'sld_shell_difference_min': 0.5,
              'sld_shell_difference_max': 3.5,
              'sld_shell_factor_min': .1,
              'sld_shell_factor_max': 2}

def make_core_shell_sphere_test(q, num_vals, rad_min = None, rad_max = None, sr_min=None, sr_max=None):
   minv, maxv = minmax(q)
   model = load_model('core_shell_sphere')
   kernel = model.make_kernel([q])
   if rad_min is None:
      radius_vals = (cs_spherep['radius_max']-cs_spherep['radius_min'])*rs(num_vals)+cs_spherep['radius_min']
   else:
      log_rad_vals = (rad_max-rad_min)*rs(num_vals)+rad_min
      radius_vals = 2**log_rad_vals
   if sr_min is None:
      sr_min = cs_spherep['shell_min']
   if sr_max is None:
      sr_max = cs_spherep['shell_max']
   shell_ratio_vals = (sr_max-sr_min)*rs(num_vals)+sr_min
   shell_vals = shell_ratio_vals*radius_vals
   core_vals = (1-shell_ratio_vals)*radius_vals
   polydispersity_vals = (shared_params['polydispersity_max'] - shared_params['polydispersity_min']) * rs(num_vals) + shared_params['polydispersity_min']
   scale_range = (shared_params['scale_max'] - shared_params['scale_min']) * rs(num_vals) + shared_params['scale_min']
   background_range = (shared_params['background_max'] - shared_params['background_min']) * rs(num_vals) + shared_params['background_min']
   sld_body_difference_vals = (shared_params['sld_body_difference_max'] - shared_params['sld_body_difference_min']) * rs(num_vals) + shared_params['sld_body_difference_min']
   sld_solvent_range = (shared_params['sld_solvent_max'] - shared_params['sld_solvent_min']) * rs(num_vals) + shared_params['sld_solvent_min']
   sld_shell_difference_vals = (cs_spherep['sld_shell_difference_max']-cs_spherep['sld_shell_difference_min'])*rs(num_vals)+cs_spherep['sld_shell_difference_min']
   sld_shell_range = sld_solvent_range-(sld_shell_difference_vals*np.random.choice([-1,1], size=num_vals))
   sld_body_range = sld_shell_range-(sld_body_difference_vals*np.random.choice([-1,1], size=num_vals))
   sphere_dict = {'radius': radius_vals, 'shell': shell_vals, 'shell ratio': shell_ratio_vals, 'core': core_vals, 'polydispersity': polydispersity_vals, 'sld_core': sld_body_range, 'sld_solvent': sld_solvent_range, 'sld_shell': sld_shell_range, 'scale': scale_range, 'background': background_range, 'contrast': sld_shell_difference_vals, 'contrast_2': sld_body_difference_vals}
   index = 0
   valid = np.zeros(num_vals)
   for qi in q:
      sphere_dict[qi] = np.zeros(num_vals)
   for index in range(num_vals):
      rad = core_vals[index]
      shell = shell_vals[index]
      pd = polydispersity_vals[index]
      scale = scale_range[index]
      background = background_range[index]
      sld_body = sld_body_range[index]
      sld_solvent = sld_solvent_range[index]
      sld_shell = sld_shell_range[index]
      if 2*rad < maxv and 2*rad > minv and shell>minv and 2*(rad+shell)<maxv:
         pars = {'radius': rad, 'radius_pd' :pd, 'radius_pd_type': 'schulz', 'radius_pd_n': 40, 'radius_pd_nsigma': 3,
            'thickness': shell,
            'sld_shell': sld_shell,
            'background': background,
            'sld_core': sld_body,
            'sld_solvent': sld_solvent,
            'scale': scale}
         Iq = call_kernel(kernel, pars)
         for qi in range(q.shape[0]):
            sphere_dict[q[qi]][index] = Iq[qi]
         valid[index]=1
   np.savetxt('valid.txt', valid)
   for key, val in sphere_dict.items():
      sphere_dict[key] = val[np.where(valid.astype(int))[0]]
   sphere_df = pandas.DataFrame(sphere_dict)
   return(sphere_df)


# Cleaning Utilities

These are a few utility functions to clean the data.
We ensure that all curves are finite and contin no NaNs. 
We check to make sure that a curve features a notable plateau, for the most part parameters are chose to be within the effective probing range determined by our choice in q vector, but we ensure that all curves show a plateau.
Lastrly we scan for artifacts that take the form of very sharp pulses, an example of which is shown here

![TRIMMED](data/trimmed_pulse.png)

In [9]:
def find_pulse(dvec):
   found = False
   for i in range(len(dvec)-1):
      if (dvec[i]<0 and dvec[i+1]>0) or (dvec[i]>0 and dvec[i+1]<0):
         if pulse_mag(dvec, i):
            found = True
   return(found)

In [10]:
def pulse_mag(dvec, ind):
   first_ind = max(ind-15, 0)
   last_ind = min(first_ind+30, len(dvec))
   absdiff = np.abs(dvec)
   meanslope = np.mean(absdiff[first_ind:last_ind])
   diffmult = 1.5
   if absdiff[ind]>(diffmult*meanslope) and absdiff[ind+1] > (diffmult*meanslope):
      return(True)
   else:
      return(False)

In [11]:
def find_cnames(df):
   columns = df.columns
   cnames = []
   for cname in columns:
      #print(cname)
      if 'Unnamed' not in cname and 'ndex' not in cname:
         cnames += [cname]
   return(cnames)

In [12]:
import types
args = types.SimpleNamespace(**{"fn": "example.csv", "target": "cs_sphere"})

In [13]:
def clean_data(filename, q):
   #args = parse_args()
   indf = pd.DataFrame(pd.read_csv(args.fn))
   #q = np.loadtxt(args.q, delimiter = ',', dtype=str)
   finite_df = find_finite(indf, q)
   non_pulse_df = trim_pulse(finite_df, q)
   valid_df = find_plateau(non_pulse_df, q)
   spec = valid_df.loc[:,q]
   valid_df.to_csv('correct_%s.csv'%(args.target))

In [14]:
def find_plateau(indf, q, tok=None):
   cnames = find_cnames(indf)
   spec = np.log10(np.array(indf.loc[:,q]))
   diff = spec[:,1:11]-spec[:,:10]
   meanslope = np.mean(diff, axis=1)
   valid_inds = np.greater(meanslope, -0.019)
   invalid_inds = np.less(meanslope, -0.019)
   where_valid = np.where(valid_inds)[0]
   #valid_inds = np.where(np.greater(meanslope, -0.01))[0]
   valid_df = indf.reindex(where_valid).reset_index(drop=True).loc[:,cnames]
   if tok is not None:
      valid_df.to_csv('find_plateau_%s.csv'%(tok))
   else:
      valid_df.to_csv('find_plateau.csv')
   vspec = np.log10(np.array(valid_df.loc[:,q]))
   return(valid_df)

In [15]:
def find_finite(indf, q, tok = None):
   cnames = find_cnames(indf)
   spec = np.array(indf.loc[:,q])
   not_finite = np.logical_not(np.isfinite(spec))
   sum_not_finite = np.sum(not_finite, axis=1)
   where_finite = np.equal(sum_not_finite, 0)
   num_not_finite = np.sum(np.logical_not(where_finite))
   invalid = np.logical_not(np.isfinite(spec))
   invalid_inds = np.sum(invalid, axis=1)
   valid_inds = np.equal(invalid_inds, 0)
   positive_inds = np.equal(np.sum(np.less(spec,0),axis=1),0)
   extra_valid_inds = np.where(np.logical_and(valid_inds, positive_inds))[0]
   valid_df = indf.reindex(extra_valid_inds).reset_index(drop=True).loc[:,cnames]
   if tok is not None:
      valid_df.to_csv('find_finite_%s.csv'%(tok))
   else:
      valid_df.to_csv('find_finite.csv')
   return(valid_df)

In [16]:
def trim_pulse(indf, q, tok=None):
   cnames = find_cnames(indf)
   spec = np.log10(np.array(indf.loc[:,q]))
   diff = spec[:,1:]-spec[:,:-1]
   pulses = np.zeros(spec.shape[0])
   for i in range(spec.shape[0]):
      if find_pulse(diff[i,:]):
         pulses[i]=1
   pulse_spec = spec[np.where(pulses.astype(int))[0],:]
   nonpulse_spec = spec[np.where(np.logical_not(pulses))[0],:]
   good_df = indf.reindex(np.where(np.logical_not(pulses))[0]).reset_index(drop=True).loc[:,cnames]
   bad_df = indf.reindex(np.where(pulses)[0]).reset_index(drop=True).loc[:,cnames]
   if tok is not None:
      good_df.to_csv('trim_pulse_%s.csv'%(tok))
      bad_df.to_csv('pulses_%s.csv'%(tok))
   else:
      good_df.to_csv('trim_pulse.csv')
      bad_df.to_csv('pulses.csv')
   newspec = good_df.loc[:,q]
   return(good_df)

# Main

In [17]:
gf_map = {'cylinder': make_cylinder_test, #These are helper functions, one for each morphology
          'disk': make_disk_test,         # They are defined at the top of this notebook
          'cs_cylinder': make_core_shell_cylinder_test, 
          'cs_disk': make_core_shell_disk_test, 
          'sphere': make_sphere_test, 
          'cs_sphere': make_core_shell_sphere_test}

def generate(morph, q, ar_min, ar_max, shell_min, shell_max, count=1000, pt=100, suffix='TRAIN'):
   token = ''.join(np.random.choice(list(string.ascii_lowercase), 6))
   generator_function = gf_map[morph]
   is_sphere = False
   if morph not in ['sphere', 'cs_sphere']:
      ar_range = np.linspace(np.log2(ar_min), np.log2(ar_max), 21)
   else:
      ar_range = np.linspace(np.log2(40.), np.log2(470.), 21)
      is_sphere = True
#   print(ar_range)
   accepted_spec = []
   if morph not in ['cs_cylinder', 'cs_disk', 'cs_sphere']:
      is_solid = True
   else:
      is_solid = False
      sr_range = np.linspace(shell_min, shell_max, 16)
      perbin = round(3000/((ar_range.shape[0]-1)*(sr_range.shape[0]-1)))
      print('CORE SHELL %d in each bin'%(perbin))
   
   accepted_spec = []
   for i in range(len(ar_range)-1):  ###Make sure each bin is appropr
      ar_min = ar_range[i]
      ar_max = ar_range[i+1]
      if is_solid:
         perbin = round((3*count)/(ar_range.shape[0]-1))
         print('SOLID %d in each bin'%(perbin))
         if not is_sphere:
            temp_df = generator_function(q, pt, ar_min = ar_min, ar_max = ar_max)
         else:
            temp_df = generator_function(q, pt, rad_min = ar_min, rad_max = ar_max)
         finite_df = find_finite(temp_df, q, tok=token)
         non_pulse_df = trim_pulse(finite_df, q, tok=token)
         valid_df = find_plateau(non_pulse_df, q, tok=token)
         selected_df = valid_df
         num_tested = 0
         while len(selected_df) < perbin and num_tested<100:
            if not is_sphere:
               temp_df = generator_function(q, pt, ar_min = ar_min, ar_max = ar_max)
            else:
               temp_df = generator_function(q, pt, rad_min = ar_min, rad_max = ar_max)
            finite_df = find_finite(temp_df, q, tok='%s_%d'%(token, num_tested))
            non_pulse_df = trim_pulse(finite_df, q, tok='%s_%d'%(token, num_tested))
            valid_df = find_plateau(non_pulse_df, q, tok='%s_%d'%(token, num_tested))
            print(len(selected_df))
            selected_df = pd.concat((selected_df, valid_df), ignore_index = True)
            print(len(selected_df))
            num_tested += 1
         print('%s %s %f - %f %d'%(morph, suffix, ar_min, ar_max, len(selected_df)))
         accepted_spec += [selected_df]
      else:
         for j in range(len(sr_range)-1):
            sr_min = sr_range[j]
            sr_max = sr_range[j+1]
            if not is_sphere:
               temp_df = generator_function(q, pt, ar_min = ar_min, ar_max = ar_max, sr_min=sr_min, sr_max=sr_max)
            else:
               temp_df = generator_function(q, pt, rad_min = ar_min, rad_max = ar_max, sr_min=sr_min, sr_max=sr_max)
            finite_df = find_finite(temp_df, q, tok=token)
            non_pulse_df = trim_pulse(finite_df, q, tok=token)
            valid_df = find_plateau(non_pulse_df, q, tok=token)
            selected_df = valid_df
            num_tested = 0
            while len(selected_df) < perbin and num_tested<100:
               if not is_sphere:
                  temp_df = generator_function(q, pt, ar_min = ar_min, ar_max = ar_max, sr_min = sr_min, sr_max = sr_max)
               else:
                  temp_df = generator_function(q, pt, rad_min = ar_min, rad_max = ar_max, sr_min = sr_min)
               finite_df = find_finite(temp_df, q, tok='%s_%d'%(token, num_tested))
               non_pulse_df = trim_pulse(finite_df, q, tok='%s_%d'%(token, num_tested))
               valid_df = find_plateau(non_pulse_df, q, tok='%s_%d'%(token, num_tested))
               print(len(selected_df))
               selected_df = pd.concat((selected_df, valid_df), ignore_index = True)
               print(len(selected_df))
               num_tested += 1
            print('%s %s %f - %f %f - %f %d'%(morph, suffix, ar_min, ar_max, sr_min, sr_max, len(selected_df)))
            accepted_spec += [selected_df]
   shinds = np.arange(len(accepted_spec[0]))
   np.random.shuffle(shinds)
   bs = round(count/len(accepted_spec))
   print('batch size %d'%(bs))
   train_df = accepted_spec[0].reindex(shinds[:bs]).reset_index()
   test_df = accepted_spec[0].reindex(shinds[bs:2*bs]).reset_index()
   val_df = accepted_spec[0].reindex(shinds[2*bs:3*bs]).reset_index()
   print('Accepted Specctra')
   print(len(accepted_spec))
   for i in range(1,len(accepted_spec)):
      shinds = np.arange(len(accepted_spec[i]))
      np.random.shuffle(shinds)
      train_df = pd.concat((train_df, accepted_spec[i].reindex(shinds[:bs]).reset_index()), ignore_index = True)
      test_df = pd.concat((test_df, accepted_spec[i].reindex(shinds[bs:2*bs]).reset_index()), ignore_index = True)
      val_df = pd.concat((val_df, accepted_spec[i].reindex(shinds[2*bs:3*bs]).reset_index()), ignore_index = True)
   #train_df.to_csv('train_%s.csv'%(args.suffix))
   #test_df.to_csv('test_%s.csv'%(args.suffix))
   #val_df.to_csv('val_%s.csv'%(args.suffix))
   #outlist = open('%s.bins'%(args.suffix),'w')
   #outlist.write('Aspect Ratio %s\n'%(', '.join(ar_range)))
   #if not is_solid:
   #   outlist.write('Shell Ratio %s\n'%(', '.join(sr_range)))
   #outlist.close()
   return(train_df, test_df)


In [21]:
q = np.loadtxt('data/q_200.txt', delimiter=',', dtype='str')
targets = ['cylinder', 'disk', 'sphere', 'cs_cylinder', 'cs_disk', 'cs_sphere']
#targets = ['cylinder', 'disk', 'sphere']
train = {}
test = {}
for t in targets:
   train[t], test[t] = generate(t, q, 2, 16, .15, .95)

SOLID 150 in each bin
77
140
140
216
cylinder TRAIN 1.000000 - 1.150000 216
SOLID 150 in each bin
75
151
cylinder TRAIN 1.150000 - 1.300000 151
SOLID 150 in each bin
66
140
140
214
cylinder TRAIN 1.300000 - 1.450000 214
SOLID 150 in each bin
68
130
130
200
cylinder TRAIN 1.450000 - 1.600000 200
SOLID 150 in each bin
79
152
cylinder TRAIN 1.600000 - 1.750000 152
SOLID 150 in each bin
74
159
cylinder TRAIN 1.750000 - 1.900000 159
SOLID 150 in each bin
70
153
cylinder TRAIN 1.900000 - 2.050000 153
SOLID 150 in each bin
80
153
cylinder TRAIN 2.050000 - 2.200000 153
SOLID 150 in each bin
77
154
cylinder TRAIN 2.200000 - 2.350000 154
SOLID 150 in each bin
70
144
144
218
cylinder TRAIN 2.350000 - 2.500000 218
SOLID 150 in each bin
73
153
cylinder TRAIN 2.500000 - 2.650000 153
SOLID 150 in each bin
63
132
132
204
cylinder TRAIN 2.650000 - 2.800000 204
SOLID 150 in each bin
67
128
128
188
cylinder TRAIN 2.800000 - 2.950000 188
SOLID 150 in each bin
64
135
135
195
cylinder TRAIN 2.950000 - 3.100

In [26]:
print(test.loc[:,"scale"])

AttributeError: 'dict' object has no attribute 'loc'

In [27]:
train['cylinder']['scale']

0      0.623455
1      0.755797
2      0.997746
3      0.533729
4      0.776448
         ...   
995    1.271009
996    0.518325
997    1.431435
998    1.140597
999    0.702690
Name: scale, Length: 1000, dtype: float64

In [19]:
for t in train.keys():
    train[t].to_csv("data/EXAMPLE_%s.csv"%(t))